## Problem Description
Consider a 5x5 grid. Write a system of equations $Ax=b$ to solve for the node voltages. Provide your answer in the form $A_{ij} = \ast$, $i=1,\ldots,25$, $j=1,\ldots,25$, $b_{i} = \ast$, $i=1,\ldots,25$.


Write a function that reads in a file to obtain the resistances on each link. The file will contain one line for each link of the form: `NODE1, NODE2, RESISTANCE`. For example, if there is a resistance of $5$ between nodes 2 and 3, then the corresponding line would be
| Node1 | Node2 | Resistance |
|-------|-------|------------|
|   2   |   3   |     5      |

In [3]:
import json

def read_resistances_json(file_name):
    resistances = {}
    with open(file_name, 'r') as f:
        data = json.load(f)
        for entry in data:
            node1 = entry["node1"]
            node2 = entry["node2"]
            resistance = entry["resistance"]
            resistances[(node1, node2)] = resistance
    return resistances

In [4]:
resistances = read_resistances_json('node_resistances.json')
print(resistances)

{(1, 2): 2.0, (1, 6): 3.0, (2, 3): 2.5, (2, 7): 3.5, (3, 4): 3.0, (3, 8): 4.0, (4, 5): 2.5, (4, 9): 3.0, (5, 10): 3.5, (6, 7): 4.0, (6, 11): 2.0, (7, 8): 2.5, (7, 12): 3.5, (8, 9): 3.0, (8, 13): 4.0, (9, 10): 2.5, (9, 14): 3.0, (10, 15): 3.5, (11, 12): 4.0, (11, 16): 2.0, (12, 13): 2.5, (12, 17): 3.5, (13, 14): 3.0, (13, 18): 4.0, (14, 15): 2.5, (14, 19): 3.0, (15, 20): 3.5, (16, 17): 4.0, (16, 21): 2.0, (17, 18): 2.5, (17, 22): 3.5, (18, 19): 3.0, (18, 23): 4.0, (19, 20): 2.5, (19, 24): 3.0, (20, 25): 3.5, (21, 22): 4.0, (22, 23): 2.5, (23, 24): 3.5, (24, 25): 3.0}


Write a function that reads in a file to obtain the set of voltages that are at fixed points. For example, if the voltage at node 3 is equal to 6.2, the corresponding line would be
| Node | Voltage |
|------|---------|
|   3  |   6.2   |

In [5]:
import json

def read_fixed_voltages_json(file_name):
    fixed_voltages = {}
    with open(file_name, 'r') as f:
        data = json.load(f)
        for entry in data:
            node = entry["node"]
            voltage = entry["voltage"]
            fixed_voltages[node] = voltage
    return fixed_voltages

In [6]:
fixed_voltages = read_fixed_voltages_json('node_voltages.json')
print(fixed_voltages)

{1: 10.0, 25: 0.0}


1. Compute the A matrix using the data read in by the previous two files

The A matrix represents the system of equations derived from Kirchhoff’s Current Law (KCL) applied to each node in the grid. We will account for the resistances between nodes and fixed voltages. 

In [17]:

import numpy as np

resistances = read_resistances_json('node_resistances.json')
fixed_voltages = read_fixed_voltages_json('node_voltages.json')

num_nodes = max(max(node1, node2) for node1, node2 in resistances.keys())

# Initialize  A 
A = np.zeros((num_nodes, num_nodes))

for (node1, node2), resistance in resistances.items():
    conductance = 1 / resistance  # Conductance is the reciprocal of resistance
    
    # Subtract conductance
    A[node1 - 1][node2 - 1] = -conductance
    A[node2 - 1][node1 - 1] = -conductance
    
    # Add conductance 
    A[node1 - 1][node1 - 1] += conductance
    A[node2 - 1][node2 - 1] += conductance

# Apply fixed voltages by modifying the A
for node, voltage in fixed_voltages.items():
    A[node - 1] = np.zeros(num_nodes)  # Zero out the row for the node with fixed voltage
    A[node - 1][node - 1] = 1  # Set the diagonal element to 1 to fix the voltage at this node

print("A matrix:")
print(A)


A matrix:
[[ 1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.        ]
 [-0.5         1.18571429 -0.4         0.          0.          0.
  -0.28571429  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.         -0.4         0.98333333 -0.33333333  0.          0.
   0.         -0.25        0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.          0.         -0.33333333  1.06666667 -0.4         0.
   0.          0.         -0.33333333  0.          0.          0.
   0.          0.    

2. Compute the LU factorization of the matrix.


In [18]:
import scipy.linalg as la

# Perform LU factorization
P, L, U = la.lu(A)

# Display the results
print("P matrix:")
print(P)
print("L matrix:")
print(L)
print("U matrix:")
print(U)


P matrix:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0

3. Compute and output the node voltages and the currents through each link.


In [19]:
import numpy as np
import scipy.linalg as la


b = np.zeros(num_nodes)

# Set the fixed voltages
for node, voltage in fixed_voltages.items():
    b[node - 1] = voltage

# Solve for node voltages using the LU decomposition method
P, L, U = la.lu(A)

# Use forward substitution
y = la.solve(L, np.dot(P, b))

# Use backward substitution
node_voltages = la.solve(U, y)

print("Node Voltages:")
for i, voltage in enumerate(node_voltages, start=1):
    print(f"Node {i}: {voltage:.2f} V")

print("\nCurrents through each link:")
currents = {}
for (node1, node2), resistance in resistances.items():
    current = (node_voltages[node1 - 1] - node_voltages[node2 - 1]) / resistance
    currents[(node1, node2)] = current
    print(f"Current between Node {node1} and Node {node2}: {current:.2f} A")


Node Voltages:
Node 1: 10.00 V
Node 2: 8.27 V
Node 3: 7.03 V
Node 4: 6.08 V
Node 5: 5.72 V
Node 6: 7.79 V
Node 7: 7.00 V
Node 8: 6.29 V
Node 9: 5.55 V
Node 10: 5.22 V
Node 11: 6.71 V
Node 12: 6.02 V
Node 13: 5.42 V
Node 14: 4.68 V
Node 15: 4.27 V
Node 16: 5.97 V
Node 17: 5.26 V
Node 18: 4.59 V
Node 19: 3.57 V
Node 20: 2.73 V
Node 21: 5.59 V
Node 22: 4.83 V
Node 23: 4.04 V
Node 24: 2.46 V
Node 25: 0.00 V

Currents through each link:
Current between Node 1 and Node 2: 0.86 A
Current between Node 1 and Node 6: 0.74 A
Current between Node 2 and Node 3: 0.50 A
Current between Node 2 and Node 7: 0.36 A
Current between Node 3 and Node 4: 0.32 A
Current between Node 3 and Node 8: 0.18 A
Current between Node 4 and Node 5: 0.14 A
Current between Node 4 and Node 9: 0.17 A
Current between Node 5 and Node 10: 0.14 A
Current between Node 6 and Node 7: 0.20 A
Current between Node 6 and Node 11: 0.54 A
Current between Node 7 and Node 8: 0.28 A
Current between Node 7 and Node 12: 0.28 A
Current between

4. Write the output of the previous three steps to a file.


In [20]:
import numpy as np
import scipy.linalg as la

#write results to a file
def write_output_to_file(A, L, U, node_voltages, currents, file_name):
    with open(file_name, 'w') as f:
        # Write A matrix
        f.write("A matrix:\n")
        f.write(np.array2string(A, precision=2, separator=', ') + "\n\n")
        
        # Write LU factorization
        f.write("L matrix:\n")
        f.write(np.array2string(L, precision=2, separator=', ') + "\n\n")
        
        f.write("U matrix:\n")
        f.write(np.array2string(U, precision=2, separator=', ') + "\n\n")
        
        # Write node voltages
        f.write("Node Voltages:\n")
        for i, voltage in enumerate(node_voltages, start=1):
            f.write(f"Node {i}: {voltage:.2f} V\n")
        f.write("\n")
        
        # Write currents each link
        f.write("Currents through each link:\n")
        for (node1, node2), current in currents.items():
            f.write(f"Current between Node {node1} and Node {node2}: {current:.2f} A\n")
        f.write("\n")


output_file_name = "circuit_analysis_output.txt"
write_output_to_file(A, L, U, node_voltages, currents, output_file_name)

print(f"Output has been written to {output_file_name}")


Output has been written to circuit_analysis_output.txt


5. Repeat the above but with a tree network (draw graph). Comment on the relative difficulty of solving for a tree as opposed to a grid.